In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [3]:
# Load data
path = '/home/han/repos/BEACO2N-CO2-Correction/data/'
X_train  = torch.Tensor(np.load(path + 'X_train.npy', allow_pickle=True))
X_test   = torch.Tensor(np.load(path + 'X_test.npy', allow_pickle=True))
X_val    = torch.Tensor(np.load(path + 'X_val.npy', allow_pickle=True))
y_train  = torch.Tensor(np.load(path + 'y_train.npy', allow_pickle=True))
y_test   = torch.Tensor(np.load(path + 'y_test.npy', allow_pickle=True))
y_val    = torch.Tensor(np.load(path + 't_val.npy', allow_pickle=True))

In [17]:
# Parameters
n_iters      = 50
nsize_in     = X_train.shape[1] 
nsize_out    = 1
nsize_hidden1 = int(X_train.shape[1]/4)
nsize_hidden2 = int(nsize_hidden1/4)
nsize_hidden3 = int(nsize_hidden2/4)
n_layers     = 4
activations = [nn.Sigmoid(),#nn.ReLU(),
               nn.Sigmoid(),#nn.ReLU(),
               nn.Sigmoid(),#nn.ReLU(), 
               None]
layer_combiners = [nn.Linear(nsize_in, nsize_hidden1),
                   nn.Linear(nsize_hidden1, nsize_hidden2),
                   nn.Linear(nsize_hidden2, nsize_hidden3),
                   nn.Linear(nsize_hidden3, nsize_out)]
learning_rate = 0.1

In [18]:
def create_model(n_layers, activation, combiners):
    
    if len(activation) != n_layers:
        act = activation[0]*(n_layers - 1)
        act.append(activation[-1])
    else:
        act = activations
    if len(combiners) != n_layers:
        combine = combiners[0]*(n_layers - 1)
        combine.append(combiners[-1])
    else:
        combine = combiners
        
    for layer in range(n_layers):
        if layer == 0:
            model = nn.Sequential(combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
        else:
            model.add_module(name=str(2*layer),module=combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
                
    return model

In [22]:
try:
    del model
except:
    pass
model = create_model(n_layers, activations, layer_combiners)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [32]:
def run(  n_iters, model=None, loss_fn=None, optimizer=None, lr=0.1,restart=False, pr=True):
    losses = torch.empty(n_iters)
    if restart:
       
        try:
            del model
        except:
            pass
        model = create_model(n_layers, activations, layer_combiners)
        loss_fn = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(),lr=lr)
    for epoch in range(n_iters):
       # Forward propagation
        y_pred = model(X_train)

        #Compute and print loss
        loss  = loss_fn(y_pred, y_train)
        losses[epoch] = loss.item()
        if pr:
            print('epoch: ',epoch, ' loss: ', loss.item())
        if torch.isnan(loss):
            break

        #Zero the gradients (so that gradients don't accumulate from epoch to epoch)
        optimizer.zero_grad()

        #perform a backward pass (backpropagation)
        loss.backward()

        # Update the parameters
        optimizer.step()
    return losses

In [34]:
losses3 = run(10, model=model, loss_fn=criterion, optimizer=optimizer)

epoch:  0  loss:  inf
epoch:  1  loss:  inf
epoch:  2  loss:  inf
epoch:  3  loss:  inf
epoch:  4  loss:  inf
epoch:  5  loss:  inf
epoch:  6  loss:  inf
epoch:  7  loss:  inf
epoch:  8  loss:  inf
epoch:  9  loss:  nan


In [86]:
# Lets Evaluate
model.eval()
print(model.training ) #Just to be sure

ypred = model(X_test)
loss  = criterion(y_pred, y_test)
print("Loss: ", loss)

False
Loss:  tensor(443.4062, grad_fn=<MseLossBackward>)


/home/han/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([6327])) that is different to the input size (torch.Size([18981, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [89]:
ypred = model(X_test)
err = torch.norm(ypred-y_test, p=2)
print(y_test[1:10])
print(ypred[1:10])
print(err)

tensor([410.6907, 441.9533, 413.5955, 415.1597, 414.4968, 413.9651, 419.0463,
        410.7672, 407.9546])
tensor([[423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161]], grad_fn=<SliceBackward>)
tensor(131980.1406, grad_fn=<NormBackward0>)


In [90]:
y_train[1:10]

tensor([433.2132, 410.1967, 465.3893, 432.4657, 417.1220, 416.7152, 413.6294,
        413.1808, 420.4663])

In [13]:
a = torch.empty(10)
a

tensor([0.0000e+00, 1.0899e+27, 7.7052e+31, 7.2148e+22, 2.5226e-18, 6.4825e-10,
        1.0072e-11, 1.2353e-08, 6.7512e-07, 4.1654e-11])

In [15]:
a[2] = 3
a

tensor([0.0000e+00, 1.0899e+27, 3.0000e+00, 7.2148e+22, 2.5226e-18, 6.4825e-10,
        1.0072e-11, 1.2353e-08, 6.7512e-07, 4.1654e-11])